### Strings and Subprocesses

---

## Overview

1. String Formatting:
    1. String operations 
    2. Olde style
    3. New style
    4. f-strings
    5. Template strings
2. Subprocesses
    1. Purpose
    2. OS dependency
    3. `subprocess`
        * `check_output`
        * `run`
        * Syntax
        * Blocking vs Non-blocking (`Popen`)

---

## String Formatting

There is *always* more than one way to do something with the same result in programming. However, there is a specific line from the Zen of Python that explains what to do in this scenario:
> There should be one-- and preferably only one --obvious way to do it.

In [1]:
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


However, when it comes to *string formatting*, there are 4 ways to do it...and each has its place.

### Setup

We will be working with a couple of variables throughout the string formatting demonstration. So, let's get them out of the way now:

In [2]:
fName = 'Sharkus'
job = 'GSI'
power = 0x2329

dict_of_types = {
    'word': 'spam',
    'integer': 42,
    'exponent': 1e12,
    'decimal': 3.14
}

### String Operations

When considering `str` objects, they have *a lot* of methods.

In [3]:
[i for i in dir(str) if not i.startswith('_')]

['capitalize',
 'casefold',
 'center',
 'count',
 'encode',
 'endswith',
 'expandtabs',
 'find',
 'format',
 'format_map',
 'index',
 'isalnum',
 'isalpha',
 'isascii',
 'isdecimal',
 'isdigit',
 'isidentifier',
 'islower',
 'isnumeric',
 'isprintable',
 'isspace',
 'istitle',
 'isupper',
 'join',
 'ljust',
 'lower',
 'lstrip',
 'maketrans',
 'partition',
 'replace',
 'rfind',
 'rindex',
 'rjust',
 'rpartition',
 'rsplit',
 'rstrip',
 'split',
 'splitlines',
 'startswith',
 'strip',
 'swapcase',
 'title',
 'translate',
 'upper',
 'zfill']

However, strings have very few *operations*

In [4]:
# Concatenation
fName + job

'SharkusGSI'

Making simple strings with concatenation makes sense, but making complete statements, log entries, or complex outputs is draining on the coder

In [5]:
greeting = 'Hello, my name is ' + fName + ', and I have ' + str(power) \
+ ''' power and currently working as a ''' + job
print(greeting)

Hello, my name is Sharkus, and I have 9001 power and currently working as a GSI


In [6]:
# Multiplicity
fName * 9

'SharkusSharkusSharkusSharkusSharkusSharkusSharkusSharkusSharkus'

In [7]:
# What about difference?
fName - job

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [8]:
# Division
fName / job

TypeError: unsupported operand type(s) for /: 'str' and 'str'

Not to be particularly rigorous, but what about `modulo` math?

In [9]:
# Modulo
fName % job

TypeError: not all arguments converted during string formatting

That's different...</br>
The modulo (`%`) is a unique operator when used on strings. It is called a *formatting* or *interpolation* operator, and its usage is a little unique.

## Ye olde style: `printf`

In [11]:
# Syntax
ex = 'My name is %s'
print(ex % fName)

My name is Sharkus


The biggest issue with `%` string formatting is that it requires the user to *declare* their types beforehand

In [14]:
# Quote
print('''In order to maintain air-speed velocity, 
a %s needs to beat its wings %f times every second, right?''' % ('swallow', 42))

In order to maintain air-speed velocity, 
a swallow needs to beat its wings 42.000000 times every second, right?


See the difference? `%s` means string, and `%d` means integer. There is a [whole list of these](https://docs.python.org/3/library/stdtypes.html#old-string-formatting). This is known as *conversion types*. Using these, the user indicates what type of object they are going to put in that place and the interpreter figures out how to convert it to a string.

### Placeholder Unpacking

In [17]:
'%(word)s %(decimal)f %(exponent)e %(integer)d' % (dict_of_types)

'spam 3.140000 1.000000e+12 42'

There is a lot to be seen in the last example. Let's walk through it.

1. Dictionary of values of different types
2. Placeholders that link to the dictionary keys
3. Conversion of types
4. naturally unpacked dictionary

The `%` acts as a placeholder, such that we can make template strings. This can be helpful in operations like SQL

In [19]:
# Greeting example: old style
greeting = 'My name is %s, and I am a %s, and I have %d power'
print(greeting % (fName, job, power))

My name is Sharkus, and I am a GSI, and I have 9001 power


### The Drawbacks
* Can be burdensome to read and write.
* Not very flexible or extensible
* Discouraged by the Python Devs:
> “The formatting operations described here exhibit a variety of quirks that lead to a number of common errors (such as failing to display tuples and dictionaries correctly).

## The 'new-style'

In [21]:
# Quote
quote = '''in order to maintain air-speed velocity, 
a {} needs to beat its wings {} times every second, right?'''
print(quote.format('sparrow', 42))

in order to maintain air-speed velocity, 
a sparrow needs to beat its wings 42 times every second, right?


What's more is that you can use the placeholders to index the arguments.

In [24]:
# Argument Unpacking
args = dict_of_types.values()

# index placeholders
idx_str = 'word = {0}, integer={3}, exponent={2}, float={1}'

idx_str.format(*args)

'word = spam, integer=3.14, exponent=1000000000000.0, float=42'

Again, I slipped in some extra here. You have seen it before, but mainly in `print` functions: `*` *unpacking*

That is cool, right? But what if you wanted to still convert the values like we did last time? We can use a 'mini-language' to control conversion and precision. The full documentation of the mini-language can be found [here](https://docs.python.org/3/library/string.html#formatspec)

In [28]:
a = [str(i) for i in range(5)]
','.join(a)

'0,1,2,3,4'

In [29]:
print(
    '\n'.join(
        [
            '{:*^10}', # Fill 
            '{:08b}',  # Binary
            '{:.2e}',  # Scientific Notation
            '{:.0f}'   # Integer
        ]  
    ).format(*args)
)

***spam***
00101010
1.00e+12
3


Like 'old-style', we can still use keyword arguments as well. Like list unpacking, we are using keyword unpacking (`**`)

In [30]:
print(
    '\n'.join(
        [
            '{word:*^10}',
            '{integer:08b}',
            '{exponent:.2e}',
            '{decimal:.0f}'
        ]
    ).format(**dict_of_types)
)

***spam***
00101010
1.00e+12
3


In [31]:
# Greeting example: new style
greeting = 'My name is {} and I am a {} and I have {} power.'
print(greeting.format(fName, job, power))

My name is Sharkus and I am a GSI and I have 9001 power.


Now, why does this matter? What could you use it for? Think of some use cases.

### Generating logs

In [32]:
import string
import uuid

template = 'item: {}, guid: {}, character: {}'
for i, s in enumerate(string.punctuation):
    print(template.format(i, uuid.uuid4(), s))

item: 0, guid: 59ebe58e-0ddf-46eb-b3a6-d4bef032656b, character: !
item: 1, guid: 3b44d0c0-38de-4b9f-9e92-4364bc02fda5, character: "
item: 2, guid: 69846599-adda-42c2-b600-f83f0e025d77, character: #
item: 3, guid: 3634b2a5-dedc-4db9-bdcc-0b9071f0e02d, character: $
item: 4, guid: 9809e37e-1a38-49ec-a3c6-6f6711b68bb0, character: %
item: 5, guid: 2c0b7c65-fd34-4665-9552-26320b994e99, character: &
item: 6, guid: a1412aa2-5135-4ab3-bad1-c75861025bac, character: '
item: 7, guid: 67ccbe74-74c1-42d4-924a-895e3963edb4, character: (
item: 8, guid: 9e4c5a6c-51b3-4213-bb97-533dd0532b22, character: )
item: 9, guid: 2deea9db-8f2e-4fde-b014-e06d0367a816, character: *
item: 10, guid: 360fe95c-b30f-402b-9936-4a5918f61eb6, character: +
item: 11, guid: cc27f981-c01b-460f-9468-309e8782f63a, character: ,
item: 12, guid: 2fe38b57-3685-48e1-8cb6-7b9bd346c3b8, character: -
item: 13, guid: 5a9c3cae-0b72-45aa-a5bd-9cfe29d5977b, character: .
item: 14, guid: f5eed65e-2342-474a-9c92-16320e504f30, character: /
item:

### The Drawbacks

1. having to write `.format()`
2. cannot call local variables instead
3. variable evaluation is 'dumb'

## The *newest-style*: f-strings

New to Python 3.6, f-strings look and feel like `{}`, but with a little extra under the hood.

In [33]:
# Greeting example: f-strings
greeting = f'My name is {fName} and I am a {job} and I have {power} power'
print(greeting)

My name is Sharkus and I am a GSI and I have 9001 power


Above shows that I no longer need to worry about the order arguments within a `.format()` method, or passing keyword arguments. I can directly reference the variables themselves.

You may ask, "Oh Senpai, what about formatting and type conversion?" and I would said, "Young padawan, there is much doubt in your voice. Still convert and format, we can"

In [36]:
# Convert and format

print(
    '\n'.join(
        [
            f'{"blah" * 9:*^10}',
            f'{dict_of_types["integer"]:08b}',
            f'{dict_of_types["exponent"]:.2e}',
            f'{dict_of_types["decimal"]:.0f}'
        ]
    )
)

blahblahblahblahblahblahblahblahblah
00101010
1.00e+12
3


### The important part

Besides just doing type conversion and formatting, there was something else a little more subtle you probably can pick up on: *expression interpolation*.

This is just a fancy way of saying that expressions within `{}` are evaluated and the output is converted into strings.

In [45]:
sv = namedtuple('sv', ('inter', 'string', 'float', 'scale', 'offset'))
some_var = sv(1, 'a', 36.3, 12, 1)

def foo(a_var):
    print( a_var.inter + a_var.float)
    print(a_var.string.upper())
    
foo(some_var)

37.3
A


In [40]:
import random
from collections import namedtuple

# bar square
Bar = namedtuple('Bar', ('x', 'square'))

In [41]:
# Explain namedtuple
please = Bar(square = 11, x = 2)
print(please)

Bar(x=2, square=11)


In [43]:
please.square

11

In [46]:
# Pass namedtuple or default
def foo(x=None):
    if x is None:
        return Bar(x,0)
    else:
        return Bar(x, x**2)

for i in range(0, 100, 10):
    print(f'{foo(random.randint(i, 100))}')

Bar(x=10, square=100)
Bar(x=41, square=1681)
Bar(x=65, square=4225)
Bar(x=58, square=3364)
Bar(x=95, square=9025)
Bar(x=83, square=6889)
Bar(x=90, square=8100)
Bar(x=76, square=5776)
Bar(x=100, square=10000)
Bar(x=92, square=8464)


In [51]:
# Capital
>>> F'this!'

'this!'

## Template Strings

To use Template strings, you must import them from the `string` module.

In [53]:
from string import Template

In [55]:
# Greeting example: Template strings
t_string = Template('''Hello, my name is $name, 
I have $power power, and currently working as a $job''')

# Template substitution
print(t_string.substitute(name=fName, power=power, job=job))

Hello, my name is Sharkus, 
I have 9001 power, and currently working as a GSI


---

## Subprocesses

One of the biggest strengths of Python is its use as a *glue* language. It can 'glue' together a bunch of programs into a highly extensible & flexible pipline.

### Purpose (Obligatory wall of text)
One of the most common, yet complicated, tasks that most programming languages need to do is spawning new processes. This could be as simple as seeing what files are present in the current working directory (`ls`) or as complicated as creating a program workflow that *pipes* output from one program into another program's input. <br/><br/>
Many such tasks are easily taken care of through the use of Python libraries and modules (`import`) that *wrap* the programs into Python code, effectively creating Application Programming Interfaces (API). <br/><br/>
However, there are many use cases that require the user to make calls to the terminal from ***within*** a Python program.

### Operating System Conundrum

First, we need to address the following issue. As many in this class have found out, while Python can be installed on most operating systems; doing the same thing in one operating system (Unix) may not always yield the same results in another (Windows).<br/><br/>
The very first step to making a program **"OS-agnostic"** is through the use of the `os` module.

In [56]:
import os

The `os` module wraps OS-specific operations into a set of standardized commands. <br/>
For instance, the Linux end-of-line (EOL) character is a `\n`, but `\r\n` in Windows. In Python, we can just use the following:

In [57]:
# EOL
os.linesep

'\n'

This above command detects the current environment, and sets the EOL it will be using based on that.

Another example, in a Linux environment, one must use the following command to list the contents of a given directory:
```
ls -alh 
```

In Windows, the equivalent is as follows:
```
dir
```

Python allows users to do a single command, in spite of the OS:

In [58]:
# Directory contents
os.listdir()

['.git',
 '.gitignore',
 '.ipynb_checkpoints',
 'age_diff.py',
 'Capture.PNG',
 'cats.jpg',
 'Comps_and_generators.ipynb',
 'datasets',
 'Data_viz.ipynb',
 'data_viz_setup.ipynb',
 'DRDs.fa',
 'Functions_ctrl_stuct.ipynb',
 'Git_tutorial.ipynb',
 'Git_tutorial.pptx',
 'mdsherm_st.py',
 'name_main_demo.py',
 'Numpy_Pandas.ipynb',
 'OOPS.ipynb',
 'README.md',
 'sqlalchemy.ipynb',
 'SQLalchemy_query.ipynb',
 'st.py',
 'st2.py',
 'string_and_sub.ipynb',
 'String_formatting.ipynb',
 'Subprocess.ipynb',
 'syntax.png',
 'test_data',
 '__pycache__']

However, the biggest issue for creating an OS-agnostic program is ***paths*** <br/>
Windows: `"C:\\Users\\MDS\\Documents"`<br/>
Linux: `/mnt/c/Users/MDS/Documents/`<br/><br/>
Enter Python:

In [59]:
# path joining from pwd
os.path.join('.', 'Users', 'MDS', 'Documents')

'./Users/MDS/Documents'

## ***NOTE***: From here on out, this notebook will *only* work on **Linux**

### `subprocess`

If you Google anything on how to run shell commands, but don't specify Python 3.x, you will likely get an answer that includes `popen`, `popen2`, or `popen3`. These were the most prolific ways to *open* a new *p*rocess. In Python 3.x, they encapsulated these functions into a new one called `run` available through the `subprocess` library.

In [60]:
# Import and alias
import subprocess as sp

The first thing we will look are trivial examples that demonstrate just capturing the *output* (stdout) of a program

#### `check_output`

In [61]:
# check_output returns a bytestring by default, so I set encoding to convert it to strings.
print(
    sp.check_output(
        [
            'ls',          # Command
            '-ahl',        # Command line arguments
            os.getcwd()    # ...
        ], 
        encoding='utf_8')  # Change from bytes to string
    
)

total 3.3M
drwxrwxrwx 1 root root 4.0K Oct 29 11:17 .
drwxrwxrwx 1 root root 4.0K Oct 22 06:40 ..
-rwxrwxrwx 1 root root  261 Sep 25 06:49 age_diff.py
-rwxrwxrwx 1 root root  24K Sep 25 06:53 Capture.PNG
-rwxrwxrwx 1 root root 855K Sep  6 09:17 cats.jpg
-rwxrwxrwx 1 root root  46K Oct 22 11:36 Comps_and_generators.ipynb
drwxrwxrwx 1 root root 4.0K Oct 19 13:36 datasets
-rwxrwxrwx 1 root root 503K Oct  8 11:08 Data_viz.ipynb
-rwxrwxrwx 1 root root  15K Oct  4 08:46 data_viz_setup.ipynb
-rwxrwxrwx 1 root root  16K Oct  5 18:12 DRDs.fa
-rwxrwxrwx 1 root root  29K Sep 26 11:32 Functions_ctrl_stuct.ipynb
drwxrwxrwx 1 root root 4.0K Oct 29 09:52 .git
-rwxrwxrwx 1 root root  175 Oct 25 07:55 .gitignore
-rwxrwxrwx 1 root root  40K Sep 10 11:30 Git_tutorial.ipynb
-rwxrwxrwx 1 root root 1.4M Sep  6 09:17 Git_tutorial.pptx
drwxrwxrwx 1 root root 4.0K Oct 24 10:15 .ipynb_checkpoints
-rwxrwxrwx 1 root root  15K Oct 23 15:50 mdsherm_st.py
-rwxrwxrwx 1 root root  375 Oct  5 16:59 name_main_demo.py
-r

However, while the `check_output` function is still in the `subprocess` module, it can easily be converted into into a more specific and/or flexible `run` function signature.

#### `run`

In [62]:
sub = sp.run(
    [
        'ls',                # The command we want to run
        '-ahl',              # Arguments for the command
        os.getcwd()          # ...
    ],
    encoding='utf_8',        # Converting byte code
    stdout=sp.PIPE,          # Where to send the output
    check=True               # Whether to raise an error if the process fails
)              

In [63]:
# Let's see what sub can do
[i for i in dir(sub) if not i.startswith('_')]

['args', 'check_returncode', 'returncode', 'stderr', 'stdout']

In [64]:
print(sub.stdout)

total 3.3M
drwxrwxrwx 1 root root 4.0K Oct 29 11:19 .
drwxrwxrwx 1 root root 4.0K Oct 22 06:40 ..
-rwxrwxrwx 1 root root  261 Sep 25 06:49 age_diff.py
-rwxrwxrwx 1 root root  24K Sep 25 06:53 Capture.PNG
-rwxrwxrwx 1 root root 855K Sep  6 09:17 cats.jpg
-rwxrwxrwx 1 root root  46K Oct 22 11:36 Comps_and_generators.ipynb
drwxrwxrwx 1 root root 4.0K Oct 19 13:36 datasets
-rwxrwxrwx 1 root root 503K Oct  8 11:08 Data_viz.ipynb
-rwxrwxrwx 1 root root  15K Oct  4 08:46 data_viz_setup.ipynb
-rwxrwxrwx 1 root root  16K Oct  5 18:12 DRDs.fa
-rwxrwxrwx 1 root root  29K Sep 26 11:32 Functions_ctrl_stuct.ipynb
drwxrwxrwx 1 root root 4.0K Oct 29 09:52 .git
-rwxrwxrwx 1 root root  175 Oct 25 07:55 .gitignore
-rwxrwxrwx 1 root root  40K Sep 10 11:30 Git_tutorial.ipynb
-rwxrwxrwx 1 root root 1.4M Sep  6 09:17 Git_tutorial.pptx
drwxrwxrwx 1 root root 4.0K Oct 24 10:15 .ipynb_checkpoints
-rwxrwxrwx 1 root root  15K Oct 23 15:50 mdsherm_st.py
-rwxrwxrwx 1 root root  375 Oct  5 16:59 name_main_demo.py
-r

The main utility of `check_output` was to capture the output (stdout) of a program. By using the `stdout=subprocess.PIPE` argument, the output can easily be captured, along with its return code. A return code signifies the program's exit status: 0 for success, anything else otherwise

In [65]:
sub.returncode

0

With our `run` code above, our program ran to completetion, exiting with status 0. The next example shows a different status.

In [66]:
print(
    sp.run(
        'exit 1',      # Command & arguments
        shell = True   # Run from the shell
    )
)

CompletedProcess(args='exit 1', returncode=1)


However, if the `check=True` argument is used, it will raise a `CalledProcessError` if your program exits with anything different than 0. This is helpful for detecting a pipeline failure, and exiting or correcting before attempting to continue computation.

In [67]:
print(
    sp.run(
        'exit 1',      # Command & arguments
        shell = True,  # Run from the shell
        check = True   # Check exit status
    )
)

CalledProcessError: Command 'exit 1' returned non-zero exit status 1.

## Syntax

Hopefully, you have picked up that I seemingly used two different syntaxes when using `run`:<br/>
1. A list of arguments: `subprocess.run(['ls', '-ahl', os.getcwd()], ...)` 
2. A string and `shell`: `subprocess.run('exit 1', shell = True, ...)`

The preferred way of using `run` is the first way. This is mainly for security purposes (to prevent shell injection attacks), but it also allows the module to take care of any required escaping and quoting of arguments for a pseudo-OS-agnostic approach. That said, some programs only work on one OS, and therefore, there is often little reason one should use `run` one way or another besides habit.

There are some guidelines though:
1. Sequence (list) of arguments is generally preferred
2. A str is appropriate if the user is just calling a program with no arguments
3. The user should use a str to pass argument if `shell` is `True`<br/>
Your next questions should be, "What is `shell`?"

`shell` is just your terminal/command prompt. This is the environment where you call `ls/dir` in. It is also where users can define variables. More importantly, this is where your *environmental variables* are set...like `PATH`.<br/><br/>
By using `shell = True`, the user can now use shell-based environmental variable expansion from within a Python program.

In [68]:
print(
    sp.run(
        'echo $PATH',            # Command
        shell = True,            # Use the shell
        stdout=sp.PIPE,          # Where to send it
        encoding='utf_8'         # Convert from bytes to string
    ).stdout.split(':')[:5]      # Look at the output
)

['/root/miniconda3/envs/teaching/bin', '/root/miniconda3/envs/teaching/bin', '/root/.cargo/bin', '/root/.local/bin', '/root/miniconda3/bin']


For the most part, you shouldn't need to use `shell` simply because Python has modules in the standard library that can do most of the shell commands. For example `mkdir` can be done with `os.mkdir()`, and `$PATH` can be found using `sys.path` 

# Blocking vs Non-blocking

The last topic of this lecture is "blocking". This is computer science lingo/jargon for whether or not a program ***waits*** until something is complete before moving on. Think of this like a really bad website that takes forever to load because it is waiting until it has rendered all its images first, versus the website that sets the formatting and text while it works on the images.

For the purposes of instruction, here are a few notes:
1. `subprocess.run()` is blocking (it waits until the process is complete)
2. `subprocess.Popen()` is non-blocking (it will run the command, then move on)

As said before, ***most*** use cases can be taken care of through the use of `run()`. However, `Popen()` allows the user a more flexible control of the subprocess call. `run()` is just a *wrapped* version of `Popen()` that simplifies use. However, `Popen()` can be used *almost* exactly the same way (albeit with more optional parameters).

An example use case for `Popen()` is if the user has some intermediate data that needs to get processed, but the output of that data doesn't necessarily affect the rest of the pipeline.

#### `Popen`

In [69]:
# Use context manager to handle process while it is running,
# and gracefully close it
with sp.Popen(
    [
        'ls',         # Command
        '-ahl',       # Command line arguments
        os.getcwd()   # ...
    ],
    encoding='utf_8', # Convert from byte to string
    stdout=sp.PIPE    # Where to send it
) as proc:            # Enclose and alias the context manager
    print(
        proc.stdout.read() # Look at the output
    )

total 3.3M
drwxrwxrwx 1 root root 4.0K Oct 29 11:23 .
drwxrwxrwx 1 root root 4.0K Oct 22 06:40 ..
-rwxrwxrwx 1 root root  261 Sep 25 06:49 age_diff.py
-rwxrwxrwx 1 root root  24K Sep 25 06:53 Capture.PNG
-rwxrwxrwx 1 root root 855K Sep  6 09:17 cats.jpg
-rwxrwxrwx 1 root root  46K Oct 22 11:36 Comps_and_generators.ipynb
drwxrwxrwx 1 root root 4.0K Oct 19 13:36 datasets
-rwxrwxrwx 1 root root 503K Oct  8 11:08 Data_viz.ipynb
-rwxrwxrwx 1 root root  15K Oct  4 08:46 data_viz_setup.ipynb
-rwxrwxrwx 1 root root  16K Oct  5 18:12 DRDs.fa
-rwxrwxrwx 1 root root  29K Sep 26 11:32 Functions_ctrl_stuct.ipynb
drwxrwxrwx 1 root root 4.0K Oct 29 09:52 .git
-rwxrwxrwx 1 root root  175 Oct 25 07:55 .gitignore
-rwxrwxrwx 1 root root  40K Sep 10 11:30 Git_tutorial.ipynb
-rwxrwxrwx 1 root root 1.4M Sep  6 09:17 Git_tutorial.pptx
drwxrwxrwx 1 root root 4.0K Oct 24 10:15 .ipynb_checkpoints
-rwxrwxrwx 1 root root  15K Oct 23 15:50 mdsherm_st.py
-rwxrwxrwx 1 root root  375 Oct  5 16:59 name_main_demo.py
-r

Furthermore, `Popen` can create background processes. As such, `Popen` has a lot more functionality than `run`

In [70]:
sub_popen = sp.Popen(
    [
        'ls',          # Command
        '-ahl',        # Command line arguments
        os.getcwd()    # ...
    ],
    encoding='utf_8',  # Convert from byte to string
    stdout=sp.PIPE     # Where to send it
)
for j in dir(sub_popen):
    if j.startswith('_'):
        pass
    else:
        print(j)

args
communicate
encoding
errors
kill
pid
poll
returncode
send_signal
stderr
stdin
stdout
terminate
text_mode
universal_newlines
wait


In [71]:
sub_popen.kill()       # Close the process

ProcessLookupError: [Errno 3] No such process